In [9]:
import nltk
from nltk import word_tokenize,sent_tokenize,pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
lemmatizer=WordNetLemmatizer()

In [229]:
senti_words = []
f=open("D:/Lex/SentiWordNet_3.0.0_20130122.txt","r",encoding="utf-8") 
all_lines=f.readlines()
for line in all_lines:
    
    w=line.split('\t')[4]
    #print(w)
    i=w.index('#')
    #print(i)
    senti_words.append(w[:i])


In [230]:
senti_words.index('failure')+1

18368

In [231]:
senti_words[57756]

'failure'

In [13]:
neg= {"aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
 "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
 "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
 "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
 "neednt", "needn't", "never","no" "none", "nope", "nor", "not", "nothing", "nowhere",
 "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
 "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
 "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite","unfulfilled"}

In [48]:
boost={"absolutely":0.5, "amazingly":0.125, "awfully":0.25, "completely":0.25, "considerably":0.125,
 "decidedly":0.25 , "deeply":0.0 , "enormously":0.25 ,
 "entirely":0.5 , "especially":0.0   , "exceptionally":0.0   , "extremely":0.625   ,
 "fabulously":0.25   , "flipping":0.0   , "flippin":0.0   ,
 "fricking":0.0   , "frickin":0.0   , "frigging":0.0   , "friggin":0.0   , "fully":0.375   , "fucking":0.125   ,
 "greatly":0.125   , "hella":0.0   , "highly":0.5   , "hugely":0.25   , "incredibly":0.25   ,
 "intensely":0.0   , "majorly":0.625   , "more":0.0  , "most":0.0   , "particularly":0.125   ,
 "purely":0.0   , "quite":0.125   , "really":0.375  , "remarkably":0.125   ,
 "so":0.0   , "substantially":0.125   ,
 "thoroughly":0.625   , "totally":0.5   , "tremendously":-0.25   ,
 "uber":0.0   , "unbelievably":0.25   , "unusually":0.0   , "utterly":0.5   ,
 "very":0.25   ,
 "almost":0.0   , "barely":-0.375   , "hardly":-0.25, "kind of":0.0   , "kinda":0.0   , "kindof":0.0   , "kind-of":0.0   ,
 "less":-0.5   , "little":-0.375   , "marginally":-0.125   , "occasionally":0.0   , "partly":0.0   ,
 "scarcely":-0.25   , "slightly":-0.25   , "somewhat":-0.125, "shoddy":-0.625,"poorly":-0.75}

In [49]:
type(boost)

dict

In [50]:
def convert_tag(tag):
    
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [232]:
def find_sentiment(word,pos):
    line_nos=[index for index, value in enumerate(senti_words) if value == word]
    lines=[all_lines[lno] for lno in line_nos]
    if len(lines)==0:
        return None
    for l in lines:
        if l[0]==pos:
            pos_score,neg_score=l.split('\t')[2],l.split('\t')[3]
            return(float(pos_score)-float(neg_score))

In [233]:
def get_sentiment(word,pos):
    second_sent=find_sentiment(word,pos)
            
    if second_sent is None:
        lemma=lemmatizer.lemmatize(word,pos)
        second_sent=find_sentiment(lemma,pos)
    return second_sent

In [234]:
nouns=open('sent_nouns.txt','r',encoding='utf-8')
nouns=nouns.read()
nouns=nouns.split()

In [235]:
tweet1='''The marginalization of the Tax Cheats which was expected post #GST will now happen in right earnest as #Ewaybill rolls out. Real benefits to most organized sector companies will be seen in FY2019'''
tweet2='''From Failed #Demonetisation to Shoddy Implementation of #GST to poorly performing #Economy to unfulfilled Promise of reduction in fuel prices to not keeping the promises of providing 2 crores jobs, BJP has wrecked a nation & has broken people’s faith '''
tweet3='''There is nothing wrong with #GST implementation but the fact is we are all losers. Injected to us by @INCIndia  Even God cannot change'''
tweet4='''So, Indian businesses are mad against GST cause now they have to pay tax and earlier they weren't?'''
tweet5='''#GST in india is total failure,it’s like nightmare for tax payers Indians..in my view it could be rolled out after #2019LSPolls after corrections ~ Dr Subramanian @Swamy39 Speaking at 14th Annual 🇮🇳India Business Conference on ‘Indian Growth @Columbia_Biz '''

In [175]:
file=open('D:/LEX/ProjectS-master/DATA/15Days/All_15.txt','r',encoding='utf-8')
A=file.read()

In [253]:
A=word_tokenize(tweet5.lower())
bigrams=nltk.ngrams(A,2)

In [254]:
score=0.0
for b in bigrams:
    b_tag=pos_tag(b)
    #print(b_tag)
    second_gram=b_tag[1]
    first_gram=b_tag[0]
    if second_gram[0] in nouns or second_gram[1].startswith('J') or second_gram[1].startswith('RBR') or second_gram[1].startswith('RBS') or second_gram[1].startswith('VBG') or second_gram[1].startswith('VBN'):
        pos=convert_tag(second_gram[1])
        
        if first_gram[0] in boost:
            score+=boost[first_gram[0]]+get_sentiment(second_gram[0],pos)
            print('Boost:',b_tag,score)
   
        if first_gram[0] in neg:
            score+=-(get_sentiment(second_gram[0],pos))
            print('Neg:',b_tag,score)
            
        elif first_gram[0] not in neg and first_gram[0] not in boost:
            try:
                score+=get_sentiment(second_gram[0],pos)
                print('Normal:',b_tag,score)
            except:
                print('not found:',second_gram[0])
print(score)
           

Normal: [('is', 'VBZ'), ('total', 'JJ')] 0.0
Normal: [('total', 'JJ'), ('failure', 'NN')] 0.125
Normal: [('like', 'IN'), ('nightmare', 'NN')] -0.125
Normal: [('be', 'VB'), ('rolled', 'VBN')] -0.125
not found: annual
Normal: [('‘', 'NN'), ('indian', 'JJ')] -0.125
-0.125


In [246]:
word='mad'
lemma = lemmatizer.lemmatize(word,pos='a')            
synsets = wn.synsets(lemma,pos='a')
synset = synsets[0]
swn_synset = swn.senti_synset(synset.name())
print(swn_synset.pos_score()) 
print(swn_synset.neg_score())

0.125
0.0


In [247]:
synsets

[Synset('huffy.s.02'),
 Synset('brainsick.s.01'),
 Synset('delirious.s.02'),
 Synset('harebrained.s.01')]

In [248]:
if word in senti_words:
    line_nos=[index+1 for index, value in enumerate(senti_words) if value == word]
    line_no=all_lines[senti_words.index(word)]
    pos_score,neg_score=line_no.split('\t')[2],line_no.split('\t')[3]
            

elif lemma in senti_words: 
    line_no=lines[senti_words.index(lemma)]
    pos_score,neg_score=line_no.split('\t')[2],line_no.split('\t')[3]
    

In [250]:
[index+1 for index, value in enumerate(senti_words) if value == 'mad']

[14272]

In [252]:
line_no

'a\t02572038\t0.375\t0.25\tmad#4 insane#2 harebrained#1\tvery foolish; "harebrained ideas"; "took insane risks behind the wheel"; "a completely mad scheme to build a bridge between two mountains"\n'